# Semantic Similarity Example

## Installations

In [1]:
# !pip install --quiet --force-reinstall prompttools

## Setup imports and API keys

First, we'll need to set our API keys. If we are in DEBUG mode, we don't need to use a real OpenAI key, so for now we'll set them to empty strings.

In [2]:
import os
os.environ['DEBUG'] = ""  # Set to "1" if you want to use debug mode.
os.environ['OPENAI_API_KEY'] = ""

Then we'll import the relevant `prompttools` modules to setup our experiment.

In [3]:
from typing import Dict, Tuple
from prompttools.harness import PromptTemplateExperimentationHarness
from prompttools.experiment import OpenAICompletionExperiment

## Run experiments

Next, we create our test inputs. For this example, we'll use a prompt template, which uses [jinja](https://jinja.palletsprojects.com/en/3.1.x/) for templating.

In [4]:
prompt_templates = ["Answer the following question: {{input}}", "Respond the following query: {{input}}"]
user_inputs = [{"input": "Who was the first president?"}, {"input": "Who was the first president of the USA?"}]

Now we can define an experimentation harness for our inputs and model. We could also pass model arguments if, for example, we wanted to change the model temperature.

In [5]:
harness = PromptTemplateExperimentationHarness(OpenAICompletionExperiment,
                                               "text-davinci-003", 
                                               prompt_templates, 
                                               user_inputs,
                                               model_arguments={"temperature": 1.0})

We can then run the experiment to get results.

In [6]:
harness.run()
harness.visualize()

,prompt,response(s),latency
0,Answer the following question: Who was the first president?,\n\nThe first president of the United States was George Washington.,1.121741
1,Answer the following question: Who was the first president of the USA?,"\n\nThe first president of the United States was George Washington, who served from",0.911955
2,Respond the following query: Who was the first president?,\n\nThe first president of the United States was George Washington. He took office,1.021080
3,Respond the following query: Who was the first president of the USA?,"\n\nThe first President of the United States was George Washington, who was inaug",1.274893


You can use the `pivot` keyword argument to view results by the template and inputs that created them.

In [7]:
harness.visualize(pivot=True)

prompt_template,Answer the following question: {{input}},Respond the following query: {{input}}
user_input,,
{'input': 'Who was the first president of the USA?'},"\n\nThe first president of the United States was George Washington, who served from","\n\nThe first President of the United States was George Washington, who was inaug"
{'input': 'Who was the first president?'},\n\nThe first president of the United States was George Washington.,\n\nThe first president of the United States was George Washington. He took office


## Evaluate the model response

To evaluate the results, we'll use a pre-built eval function. We can use semantic distance to check if the model's response is similar to our expected output.

In [8]:
from prompttools.utils import similarity

In [9]:
harness.evaluate("similar_to_expected", similarity.evaluate, expected="George Washington")

Finally, we can visualize the results.

In [10]:
harness.visualize()

,prompt,response(s),latency,similar_to_expected
0,Answer the following question: Who was the first president?,\n\nThe first president of the United States was George Washington.,1.121741,0.275267
1,Answer the following question: Who was the first president of the USA?,"\n\nThe first president of the United States was George Washington, who served from",0.911955,0.344860
2,Respond the following query: Who was the first president?,\n\nThe first president of the United States was George Washington. He took office,1.021080,0.341740
3,Respond the following query: Who was the first president of the USA?,"\n\nThe first President of the United States was George Washington, who was inaug",1.274893,0.336985
